In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
sys.path.insert(1, str(Path.cwd().parent))
str(Path.cwd().parent)

'/home/ubuntu/varios/skforecast'

In [2]:
# simulate tiem series
import numpy as np
import pandas as pd
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from sklearn.preprocessing import StandardScaler

In [3]:
# simulate tiem series
import numpy as np
import pandas as pd

y = np.random.normal(0, 1, 1000)
y = pd.Series(y, index=pd.date_range('2020-01-01', periods=1000, freq='D'), name='users')
y

2020-01-01   -1.165125
2020-01-02    0.279394
2020-01-03   -0.361729
2020-01-04   -0.633743
2020-01-05    0.058005
                ...   
2022-09-22   -1.697567
2022-09-23    2.925059
2022-09-24    0.445889
2022-09-25   -0.583245
2022-09-26    0.671056
Freq: D, Name: users, Length: 1000, dtype: float64

In [4]:
y_frame = y.to_frame(name=y.name if y.name is not None else 'y')
y_frame

,users
2020-01-01,-1.165125
2020-01-02,0.279394
2020-01-03,-0.361729
2020-01-04,-0.633743
2020-01-05,0.058005
...,...
2022-09-22,-1.697567
2022-09-23,2.925059
2022-09-24,0.445889
2022-09-25,-0.583245


In [5]:
from skforecast.utils import transform_dataframe
from skforecast.utils import transform_series
from skforecast.utils import preprocess_y

In [6]:
transformer_y = StandardScaler()
y_transformed = transform_series(
                    series            = y,
                    transformer       = transformer_y,
                    fit               = True,
                    inverse_transform = False
                )
y_values, y_index = preprocess_y(y=y_transformed)

y_inverse = transform_series(
                series            = y_transformed,
                transformer       = transformer_y,
                fit               = False,
                inverse_transform = True
            )

In [7]:
transformer_y_df = StandardScaler()
y_frame_transformed = transform_dataframe(
                df                = y_frame,
                transformer       = transformer_y_df,
                fit               = True,
                inverse_transform = False
            )
y_frame_values, y_frame_index = preprocess_y(y=y_frame_transformed)
y_frame_inverse = transform_dataframe(
                df                = y_frame_transformed,
                transformer       = transformer_y_df,
                fit               = False,
                inverse_transform = True
            )

In [13]:
pd.testing.assert_series_equal(y_transformed, y_frame_transformed[y.name])
pd.testing.assert_index_equal(y_index, y_frame_index)
pd.testing.assert_series_equal(y_inverse, y_frame_inverse[y.name])